<a href="https://colab.research.google.com/github/gwohlgen/colab/blob/master/SW_Course_2019_unit10_relation_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install SPARQLWrapper


In [0]:
!git clone https://github.com/thunlp/OpenNRE.git

Cloning into 'OpenNRE'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 589 (delta 0), reused 0 (delta 0), pack-reused 585
Receiving objects: 100% (589/589), 266.64 MiB | 39.29 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [0]:
!mkdir data/our_test -p

In [0]:
import hashlib
from SPARQLWrapper import SPARQLWrapper, JSON


sparql = SPARQLWrapper("http://dbpedia.org/sparql")

data = []
  
sparql.setQuery("""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX dbo: <http://dbpedia.org/ontology/>
      PREFIX dbp: <http://dbpedia.org/resource/>
      PREFIX foaf: <http://xmlns.com/foaf/0.1/>

      SELECT ?p ?pl ?bpl ?dpl ?abstract
      WHERE { 
      ?p rdf:type dbo:Person .
      ?p rdfs:label ?pl .
      ?p dbo:abstract ?abstract .
      ?p dbo:birthPlace ?bp .
      ?p dbo:deathPlace ?dp .
      ?bp rdf:type dbo:City .
      ?dp rdf:type dbo:City .
      ?bp rdfs:label ?bpl .
      ?dp rdfs:label ?dpl .
      FILTER (?bp != ?dp) 
      filter contains(?abstract,?bpl)
      filter contains(?abstract,?dpl)
      filter contains(?abstract,?pl)
      FILTER(LANG(?abstract)='en')
      FILTER(LANG(?pl)='en')
      FILTER(LANG(?bpl)='en')
      FILTER(LANG(?dpl)='en')
      FILTER(strlen(?abstract)<500)
      } LIMIT 1000
 """) 

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
  print(result["p"]["value"])
  print(result["pl"]["value"])
  print(result["bpl"]["value"])
  print(result["dpl"]["value"])
  print()
      
  item1 = {}
  item1['sentence'] = result["abstract"]["value"]
  item1['head'] = {'word': result["pl"]["value"], 'id': hashlib.md5(result["pl"]["value"].encode('utf-8')).hexdigest()}
  item1['tail'] = {'word': result["bpl"]["value"], 'id': hashlib.md5(result["bpl"]["value"].encode('utf-8')).hexdigest()}
  item1['relation'] = 'birthPlace'

  item2 = {}
  item2['sentence'] = result["abstract"]["value"]
  item2['head'] = {'word': result["pl"]["value"], 'id': hashlib.md5(result["pl"]["value"].encode('utf-8')).hexdigest()}
  item2['tail'] = {'word': result["dpl"]["value"], 'id': hashlib.md5(result["dpl"]["value"].encode('utf-8')).hexdigest()}
  item2['relation'] = 'deathPlace'

  data.append(item1)
  data.append(item2)
  

http://dbpedia.org/resource/Theodoret
Theodoret
Antioch
Cyrrhus

http://dbpedia.org/resource/Tarab_Abdul_Hadi
Tarab Abdul Hadi
Jenin
Cairo

http://dbpedia.org/resource/Sergei_Zhurikov
Sergei Zhurikov
Sevastopol
Sloviansk

http://dbpedia.org/resource/Yehoshafat_Harkabi
Yehoshafat Harkabi
Haifa
Jerusalem

http://dbpedia.org/resource/Dmitry_Mikhailovich_Golitsyn
Dmitry Mikhailovich Golitsyn
Turku
Vienna

http://dbpedia.org/resource/Giovanni_Battista_Ruggieri
Giovanni Battista Ruggieri
Bologna
Rome

http://dbpedia.org/resource/Arconovaldo_Bonaccorsi
Arconovaldo Bonaccorsi
Bologna
Rome

http://dbpedia.org/resource/Nerio_Bernardi
Nerio Bernardi
Bologna
Rome

http://dbpedia.org/resource/Claudio_Cavazza
Claudio Cavazza
Bologna
Rome

http://dbpedia.org/resource/Baldassare_Croce
Baldassare Croce
Bologna
Rome

http://dbpedia.org/resource/Petronio_Franceschini
Petronio Franceschini
Bologna
Venice

http://dbpedia.org/resource/Obdulio_Diano
Obdulio Diano
Bernal, Argentina
Mar del Plata

http://dbped



```
[
    {
        'sentence': 'Bill Gates is the founder of Microsoft .',
        'head': {'word': 'Bill Gates', 'id': 'm.03_3d', ...(other information)},
        'tail': {'word': 'Microsoft', 'id': 'm.07dfk', ...(other information)},
        'relation': 'founder'
    },
    ...
]
```



In [0]:
from pprint import pprint
print(len(data))
pprint(data[10:20])


2000
[{'head': {'id': '1f7573b7168a9a9d330ae647623ddcdf',
           'word': 'Giovanni Battista Ruggieri'},
  'relation': 'birthPlace',
  'sentence': 'Giovanni Battista Ruggieri (died 1640) was an Italian painter '
              'of the Baroque period. He was born in Bologna. In 1606 he '
              'became a pupil of Domenichino. He was also called Battistino '
              'del Gessi, because later he became a pupil of Francesco Gessi. '
              'Ruggieri accompanied Gessi to Naples during the time of Urban '
              'VIII. He was patronized by the Giustiniani family and died in '
              'Rome in 1640. His brother Ercole was also educated by Gessi.',
  'tail': {'id': '7014d0181914fb4722b90a49d53f5471', 'word': 'Bologna'}},
 {'head': {'id': '1f7573b7168a9a9d330ae647623ddcdf',
           'word': 'Giovanni Battista Ruggieri'},
  'relation': 'deathPlace',
  'sentence': 'Giovanni Battista Ruggieri (died 1640) was an Italian painter '
              'of the Baroque pe

In [0]:
import json
with open('data/our_test/train.json', 'w') as outfile:  
    json.dump(data[:1800], outfile)
with open('data/our_test/test.json', 'w') as outfile:  
    json.dump(data[1800:], outfile)
   
    

In [0]:
!mv rel2id.json data/our_test
!mv word_vec.json data/our_test
!ls -l data/our_test


total 196088
-rw-r--r-- 1 root root        58 May 15 10:16 rel2id.json
-rw-r--r-- 1 root root    122284 May 15 10:15 test.json
-rw-r--r-- 1 root root    460658 May 15 10:15 train.json
-rw-r--r-- 1 root root 200196954 May 15 10:34 word_vec.json


In [0]:
#!mv OpenNRE/* .
#!mv rel2id.json data/our_test
!cat data/our_test/train.json

[{"sentence": "Theodoret of Cyrus or Cyrrhus (Greek: \u0398\u03b5\u03bf\u03b4\u03ce\u03c1\u03b7\u03c4\u03bf\u03c2 \u039a\u03cd\u03c1\u03c1\u03bf\u03c5; c. AD 393 \u2013 c. 458/466) was an influential theologian of the School of Antioch, biblical commentator, and Christian bishop of Cyrrhus (423\u2013457). He played a pivotal role in several 5th-century, Byzantine Church controversies that led to various ecumenical acts and schisms. He is called \"blessed\" in the Eastern Orthodox Church, and some Chalcedonian and East Syrian Christians regard him as a saint.", "head": "Theodoret", "tail": "Cyrrhus", "relation": "birthPlace"}, {"sentence": "Theodoret of Cyrus or Cyrrhus (Greek: \u0398\u03b5\u03bf\u03b4\u03ce\u03c1\u03b7\u03c4\u03bf\u03c2 \u039a\u03cd\u03c1\u03c1\u03bf\u03c5; c. AD 393 \u2013 c. 458/466) was an influential theologian of the School of Antioch, biblical commentator, and Christian bishop of Cyrrhus (423\u2013457). He played a pivotal role in several 5th-century, Byzantine C

In [0]:
!python train_demo.py our_test pcnn att

Pre-processed files exist. Loading them...
Finish loading
Total relation fact: 400
Pre-processed files exist. Loading them...
Finish loading
Total relation fact: 100
Start training...
2019-05-15 11:11:47.091728: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-15 11:11:47.092008: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x24191e0 executing computations on platform Host. Devices:
2019-05-15 11:11:47.092045: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-15 11:11:47.260805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-15 11:11:47.261386: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2419e40 executing computations on platform CUDA. Devices:
2019-05-15 11:11:47.261422: I tensorflow/compiler

In [0]:
!python draw_plot.py checkpoint/our_test_pcnn_att

Traceback (most recent call last):
  File "draw_plot.py", line 34, in <module>
    main()
  File "draw_plot.py", line 15, in main
    x = np.load(os.path.join(result_dir, model +'_x' + '.npy')) 
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/npyio.py", line 422, in load
    fid = open(os_fspath(file), "rb")
FileNotFoundError: [Errno 2] No such file or directory: './test_result/checkpoint/our_test_pcnn_att_x.npy'


In [0]:
!python test_demo.py our_test pcnn att

Pre-processed files exist. Loading them...
Finish loading
Total relation fact: 400
Pre-processed files exist. Loading them...
Finish loading
Total relation fact: 100
Testing...
2019-05-15 11:28:01.738520: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-15 11:28:01.739120: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x16911e0 executing computations on platform Host. Devices:
2019-05-15 11:28:01.739167: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-15 11:28:01.985852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-15 11:28:01.986420: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1691e40 executing computations on platform CUDA. Devices:
2019-05-15 11:28:01.986462: I tensorflow/compiler/xla/se